# Recommenders 3 -- Sequence Recommenders (45m) 

## Goals of this practical:

- Understand the sequence recommendation framework (~5min)
- Load/Format dataset (~5min)
- Understand/train the prod2vec model (~10min)
- Evaluate (~10min)
- Visualize (~10min)
- Fiddle (~5min)



In [1]:
! pip install gensim --upgrade

     |████████████████████████████████| 24.2 MB 25.9 MB/s eta 0:00:01   |██████▌                         | 4.9 MB 3.4 MB/s eta 0:00:06
     |████████████████████████████████| 119 kB 26.1 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-4.2.0-py3-none-any.whl size=109632 sha256=18b2e036926b0dccf20b85795a2b562715c13c9be73481114863346860281830
  Stored in directory: /Users/mac/Library/Caches/pip/wheels/24/f6/ea/70a0761bdfaeacff66662751fe71920e25c4c43d97098a3886
Successfully built smart-open


In [2]:
import pandas as pd
import numpy as np

# Sequence Recommenders:

> What will you click next ?

The sequence recommendation setting is a particular case of the implicit collaborative filtering setting. Given a sequence of items $i_0,i_1,...,i_n$ the goal is to predict the $i_{(n+1)},...$ items the user will consume. Playlist continuation is a neat use case of sequence recommenders. You've been listening to those songs, what can you listen to now ?


This setting differs from the classical collaborative filtering because the history is the recent trace and not the full saved interactions. Also, it's possible to do sequence recommendation without any specific latent user profile. 

#### Here we propose to explore this unpersonalized sequence recommandation

## Data used : [smallest movie-lens dataset](https://grouplens.org/datasets/movielens/)

Here we'll use the same data as before but instead of seeing $(user,item,rating)$ triplets or a $(user,item)$ interaction , we'll see item sequences: $user: [item, item,...]$

## Loading Data (same as before but in chronological order):

In [3]:
ratings = pd.read_csv("dataset/ratings.csv")
ratings = ratings.sort_values("timestamp",ascending=True)
print(ratings.iloc[0]["timestamp"] < ratings.iloc[-1]["timestamp"] ) # just checking 


# we also load titles and create an id2title dictionnary
titleCSV = pd.read_csv("dataset/movies.csv")
titleCSV.head(5)
id2title = titleCSV[["movieId","title"]].set_index("movieId").to_dict()["title"]




True


## (a) Create sequence datasets:
For this task, we need sequences of items as data:

## (Todo): extract all movie sequences (in chronological order) from the dataset:


In this dataset, each user has seen at least 20 movies.


- We need to extract all movie rating sequences (there is one per user) from the dataset:

`sequence_of_movies = [[movieid,...],[movieid,...],...]`

In [4]:
sequences_of_movies =  list(ratings.groupby(["userId"])["movieId"].apply(list))
print(sequences_of_movies)

[[804, 1210, 2628, 2826, 2018, 3578, 3617, 3744, 101, 441, 2858, 1473, 2997, 235, 1060, 356, 2700, 1500, 223, 3243, 2395, 3253, 1517, 1580, 1732, 3450, 2596, 333, 543, 231, 1042, 3052, 216, 500, 3809, 1777, 3, 2542, 2502, 1080, 1136, 1793, 1256, 1927, 923, 3671, 260, 1097, 1073, 2872, 2174, 2093, 2005, 1967, 2161, 367, 2797, 2193, 3479, 2143, 2105, 2054, 2253, 673, 3489, 1009, 1920, 3439, 3440, 1198, 1196, 3793, 1214, 2692, 1197, 3062, 2944, 2028, 2571, 2949, 1222, 1291, 1220, 457, 3703, 940, 110, 1954, 2947, 2948, 2000, 6, 2529, 2993, 2427, 3527, 3639, 2991, 592, 1275, 2916, 1573, 1408, 316, 2406, 2273, 2528, 3441, 1587, 480, 423, 3729, 2640, 733, 1049, 2058, 70, 3740, 2366, 919, 3168, 2987, 1127, 2414, 2115, 590, 2046, 349, 648, 2617, 2470, 2099, 362, 2641, 2450, 1676, 1552, 552, 1031, 736, 2094, 1377, 1023, 1, 2899, 2761, 1282, 1025, 3034, 2137, 2139, 2048, 1032, 2141, 2078, 661, 2096, 596, 2090, 1029, 1024, 2116, 4006, 2033, 1030, 50, 608, 1213, 1617, 1089, 296, 2268, 2580, 1396, 2

## (Todo): Create a train/test dataset

Here, we propose as task to predict the last 5 items of each sequence.

In [5]:
train_seq,test_seq = [],[]

for seq in sequences_of_movies:
    train_seq.append(seq[:-5])
    test_seq.append(seq[-5:])
    
last_consumed_item = [seq[-1] for seq in train_seq] # We save the last consumed item for each list
                                                    # We'll use it as a starting point

## (Todo): Create the list of the most popular movies

- Here, popular is the number of times the movie appears in a list

In [6]:
from collections import Counter

counts = ratings .groupby("movieId").count()
most_popular = counts.sort_values(by=['rating'],ascending=False).index.values
print(most_popular)
num_items = len(most_popular)

[   356    318    296 ...  58351   4083 193609]


``` python
#Most popular looks like this:
[356,318,296,2571,593,260,480,110,589,...]
 ```

## Word2Vec skip-gram <=> Prod2Vec


### Word2Vec

The MAIN idea of word2vec is to maximise the similarity (dot product) between the vectors for words which appear close together (in the context of each other) in text, and minimise the similarity of words that do not. 

This can be applied to products instead of words: it clusters similar products together.


#### Paper Abstract:
> In recent years online advertising has become increasingly ubiquitous and effective. Advertisements shown to visitors fund sites and apps that publish digital content, manage social networks, and operate e-mail services. Given such large variety of internet resources, determining an appropriate type of advertising for a given platform has become critical to financial success. Native advertisements, namely ads that are similar in look and feel to content, have had great success in news and social feeds. However, to date there has not been a winning formula for ads in e-mail clients. In this paper we describe a system that leverages user purchase history determined from e-mail receipts to deliver highly personalized product ads to Yahoo Mail users. We propose to use a novel neural language-based algorithm specifically tailored for delivering effective product recommendations, which was evaluated against baselines that included showing popular products and products predicted based on co-occurrence. We conducted rigorous offline testing using a large-scale product purchase data set, covering purchases of more than 29 million users from 172 e-commerce websites. Ads in the form of product recommendations were successfully tested on online traffic, where we observed a steady 9% lift in click-through rates over other ad formats in mail, as well as comparable lift in conversion rates. Following successful tests, the system was launched into production during the holiday season of 2014

[Prod2Vec Model](https://arxiv.org/abs/1606.07154)



## Gensim has the best python implementation of word2vec's algorithms:

We can just use these raw implementations. The only thing to do is to consider items as words:

In [7]:
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)



train_seq_str = [list(map(str,seq)) for seq in train_seq] # we just say that our items id's are strings..
    

# the following configuration is the default configuration
w2v = gensim.models.word2vec.Word2Vec(sentences=train_seq_str,
                                size=50, window=10,               ### here we train a cbow model 
                                min_count=0,                      
                                sample=0.001, ns_exponent=0.75, workers=10,
                                sg=1, hs=0, negative=15,          ### set sg to 1 to train a sg model => Prod2Vec
                                cbow_mean=0,
                                iter=50)

2021-03-23 22:18:40,047 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2021-03-23 22:18:40,049 : INFO : collecting all words and their counts
2021-03-23 22:18:40,050 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-03-23 22:18:40,081 : INFO : collected 9616 word types from a corpus of 97786 raw words and 610 sentences
2021-03-23 22:18:40,082 : INFO : Loading a fresh vocabulary
2021-03-23 22:18:40,115 : INFO : effective_min_count=0 retains 9616 unique words (100% of original 9616, drops 0)
2021-03-23 22:18:40,116 : INFO : effective_min_count=0 leaves 97786 word corpus (100% of original 97786, drops 0)
2021-03-23 22:18:40,149 : INFO : deleting the raw counts dictionary of 9616 items
2021-03-23 22:18:40,149 : INFO : sample=0.001 downsamples 5 most-common words
2021-03-23 22:18:40,150 : INFO : downsampling leaves estimated 97657 word corpus (99.9% of prior 97786)
2021-03-23 22:18:40,183 : INFO : estimated required memory fo

2021-03-23 22:18:51,556 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-23 22:18:51,620 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-23 22:18:51,633 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-23 22:18:51,650 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-23 22:18:51,653 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-23 22:18:51,659 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-03-23 22:18:51,677 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-03-23 22:18:51,698 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-03-23 22:18:51,700 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-23 22:18:51,701 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-23 22:18:51,703 : INFO : EPOCH - 7 : training on 97786 raw words (97653 effective 

2021-03-23 22:19:03,213 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-23 22:19:03,228 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-23 22:19:03,231 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-23 22:19:03,248 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-03-23 22:19:03,260 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-03-23 22:19:03,266 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-03-23 22:19:03,268 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-23 22:19:03,280 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-23 22:19:03,283 : INFO : EPOCH - 14 : training on 97786 raw words (97651 effective words) took 1.4s, 69696 effective words/s
2021-03-23 22:19:04,464 : INFO : EPOCH 15 - PROGRESS: at 10.82% examples, 8575 words/s, in_qsize 9, out_qsize 1
2021-03-23 22:19:04,464 : I

2021-03-23 22:19:11,156 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-23 22:19:11,163 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-03-23 22:19:11,167 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-03-23 22:19:11,172 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-03-23 22:19:11,173 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-23 22:19:11,179 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-23 22:19:11,180 : INFO : EPOCH - 21 : training on 97786 raw words (97658 effective words) took 1.1s, 85000 effective words/s
2021-03-23 22:19:12,523 : INFO : EPOCH 22 - PROGRESS: at 2.46% examples, 6745 words/s, in_qsize 9, out_qsize 1
2021-03-23 22:19:12,526 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-23 22:19:12,544 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-23 22:19:12,571 : IN

2021-03-23 22:19:20,469 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-03-23 22:19:20,492 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-03-23 22:19:20,494 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-23 22:19:20,499 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-23 22:19:20,501 : INFO : EPOCH - 28 : training on 97786 raw words (97660 effective words) took 1.3s, 77274 effective words/s
2021-03-23 22:19:21,557 : INFO : EPOCH 29 - PROGRESS: at 2.46% examples, 8627 words/s, in_qsize 9, out_qsize 1
2021-03-23 22:19:21,558 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-23 22:19:21,592 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-23 22:19:21,597 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-23 22:19:21,604 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-23 22:19:21,620 : IN

2021-03-23 22:19:28,409 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-23 22:19:28,436 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-03-23 22:19:28,437 : INFO : EPOCH - 35 : training on 97786 raw words (97675 effective words) took 1.2s, 80879 effective words/s
2021-03-23 22:19:29,514 : INFO : EPOCH 36 - PROGRESS: at 2.46% examples, 8406 words/s, in_qsize 9, out_qsize 1
2021-03-23 22:19:29,515 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-23 22:19:29,551 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-23 22:19:29,554 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-23 22:19:29,568 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-23 22:19:29,581 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-23 22:19:29,593 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-03-23 22:19:29,602 : IN

2021-03-23 22:19:36,697 : INFO : EPOCH - 42 : training on 97786 raw words (97632 effective words) took 1.2s, 82213 effective words/s
2021-03-23 22:19:37,797 : INFO : EPOCH 43 - PROGRESS: at 2.46% examples, 8261 words/s, in_qsize 9, out_qsize 1
2021-03-23 22:19:37,798 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-23 22:19:37,838 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-23 22:19:37,850 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-23 22:19:37,862 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-23 22:19:37,867 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-23 22:19:37,878 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-03-23 22:19:37,892 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-03-23 22:19:37,893 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-03-23 22:19:37,899 : IN

2021-03-23 22:19:48,116 : INFO : EPOCH 50 - PROGRESS: at 2.46% examples, 5446 words/s, in_qsize 9, out_qsize 1
2021-03-23 22:19:48,116 : INFO : worker thread finished; awaiting finish of 9 more threads
2021-03-23 22:19:48,148 : INFO : worker thread finished; awaiting finish of 8 more threads
2021-03-23 22:19:48,149 : INFO : worker thread finished; awaiting finish of 7 more threads
2021-03-23 22:19:48,153 : INFO : worker thread finished; awaiting finish of 6 more threads
2021-03-23 22:19:48,160 : INFO : worker thread finished; awaiting finish of 5 more threads
2021-03-23 22:19:48,185 : INFO : worker thread finished; awaiting finish of 4 more threads
2021-03-23 22:19:48,206 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-03-23 22:19:48,207 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-03-23 22:19:48,209 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-03-23 22:19:48,217 : INFO : worker thread finished; awaiting fini

### A few things:

In [9]:
print(w2v.wv.vectors[0])              # The vector of index 0
print(w2v.wv.index2word[0])           # codes for the movieId 356
print(w2v.wv.vocab["356"].index)      # Inverse mapping


[-0.04323157 -0.05270474 -0.34007475  0.14205082 -0.24318299 -0.3057525
 -0.26680902 -0.10024821  0.12646629 -0.19419484 -0.00800263  0.5085645
 -0.55438703 -0.11221337  0.1798884   0.14747529 -0.3388722  -0.33498314
 -0.5029714  -0.6262385  -0.41028562 -0.37285534  1.2811016   0.47068417
  0.03973944  0.18417193  0.14051881 -0.13924046  0.12017153  0.66925186
 -0.38512892  0.19410865 -0.39149997 -0.12952575  0.31932074 -0.4873704
  0.26252928 -0.6449865  -0.36669722 -0.35634255  0.18064493  0.32854924
 -0.14764042  0.12701704 -0.06295091 -0.3097362  -0.42474014  0.65296227
  0.48609447 -0.00340491]
356
0


## Getting similar items:

The heart of the algorithm is in the similar item search. As in word2vec, we simply use cosine distance between items to find "similar items"

### We can search by id's

In [10]:
def get_similar_ids(w2vmodel,iid,num=5):
    
    if str(iid) in w2vmodel.wv.vocab:
        return [int(iid) for iid,_ in w2vmodel.wv.most_similar(str(iid),topn=num)] 
    else:
        return []

get_similar_ids(w2v,last_consumed_item[0],num=5)

2021-03-23 22:20:31,270 : INFO : precomputing L2-norms of word weight vectors


[340, 26142, 996, 66915, 1458]

### Or by vector

In [11]:
w2v.wv[str(last_consumed_item[0])]

array([ 0.08657522,  0.8393416 ,  0.10357846,  0.01453648, -0.934053  ,
       -0.25804794,  0.00577203,  0.35194573,  0.41671062, -0.7315626 ,
        0.89869094,  0.4709449 ,  0.1799989 , -0.27718282, -0.04602511,
        0.36043477, -0.0904432 , -0.17757459,  0.02882305, -2.000345  ,
       -0.07743907,  0.43987867,  0.7453763 ,  0.81554604, -0.19746265,
       -0.45110446, -0.8679626 , -0.4026603 ,  0.42805666,  0.4598513 ,
        0.38395166,  0.3935371 , -0.55933535, -0.22194864,  0.3511198 ,
        0.6390012 ,  1.0357039 , -0.75012344, -0.22040273, -0.84606683,
        0.5156977 ,  0.69632024,  1.6565629 , -0.24527831, -1.2748258 ,
       -0.3677933 , -0.57804936, -0.04006956, -0.50069493, -0.05853381],
      dtype=float32)

In [12]:
def get_similar_vectors(w2vmodel,vec,num=5):
        return [int(iid) for iid,_ in w2vmodel.wv.most_similar(positive=[vec],topn=num)] 

get_similar_vectors(w2v,w2v.wv[str(last_consumed_item[0])],num=5) # items are strings

[157, 340, 26142, 996, 66915]

### Let's see if this works

We can query by id

In [13]:
ID = 1
NUM_SIM = 3

print("Movies similar to: ", id2title[ID])
print("")
for x in get_similar_ids(w2v,ID,NUM_SIM):
    print("--> ",id2title[x])

Movies similar to:  Toy Story (1995)

-->  Forrest Gump (1994)
-->  Aladdin (1992)
-->  Beauty and the Beast (1991)


We can also query by vector

**NOTE:** the 1st results can be the item(s) you've used to query

In [14]:
ID = 1
NUM_SIM = 4
print("Movies similar to: ", id2title[ID])
print("")
for x in get_similar_vectors(w2v,w2v.wv[str(ID)],NUM_SIM):
    print("--> ",id2title[x])

Movies similar to:  Toy Story (1995)

-->  Toy Story (1995)
-->  Forrest Gump (1994)
-->  Aladdin (1992)
-->  Beauty and the Beast (1991)


Our result was the following (for ID = 1 & NUM_SIM = 3)

>Movies similar to:  Toy Story (1995)
>-  Beauty and the Beast (1991)
-   Toy Story 2 (1999)
-   Lion King, The (1994)


Using vectors enables operations like additions to be made

In [15]:
ID1 = 2571
ID2 = 589
NUM_SIM = 10

vec = np.max([w2v.wv[str(ID1)],w2v.wv[str(ID2)]],axis=0)# + w2v.wv[str(318)]

print("Movies similar to: ", id2title[ID1] , "+",  id2title[ID2] )
print("")
for x in get_similar_vectors(w2v,vec ,NUM_SIM):
    print("--> ",id2title[x])

Movies similar to:  Matrix, The (1999) + Terminator 2: Judgment Day (1991)

-->  Matrix, The (1999)
-->  Terminator 2: Judgment Day (1991)
-->  Star Wars: Episode V - The Empire Strikes Back (1980)
-->  Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
-->  Speed (1994)
-->  Jurassic Park (1993)
-->  Star Wars: Episode IV - A New Hope (1977)
-->  Primeval (2007)
-->  Terminator, The (1984)
-->  Star Wars: Episode VI - Return of the Jedi (1983)


Ok, we now have a good base for our sequence recommendation algorithm, let's write something to evaluate our predictions

## (Todo) write a `get_relevance_list(proposed_ids,real_ids)` function:

This function will be used to compare proposed items w/ real items:


- A relevant item is an item which is in the ground truth
- It returns a list which length is the number of proposed items filled of 0's and 1's : 0 means the item is not relevant, 1 means it's relevant.

- get_relevance_list([1,2,3,4],[1,4,5,6]) should returns [1,0,0,1]  because items 1 and 4 are relevant.


In [16]:
def get_relevance_list(proposed_ids,real_ids):
    real_ids = set(real_ids)
    return [1 if proposed_ids[i] in real_ids else 0 for i in range(len(proposed_ids))]
get_relevance_list([1,2,3,4],[1,4,5,6]) #returns [1,0,0,1]

[1, 0, 0, 1]

### Let's test our function on our data

In [17]:
get_relevance_list(most_popular[:25],test_seq[1])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [18]:
get_relevance_list(get_similar_ids(w2v,last_consumed_item[0],25),test_seq[0])

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

## Ok, now, let's write prediction funtions:

- `predict_pop` will recommend the k's most popular items
- `predict_w2v` will recommend the k's most similar items to the last one consumed

#### (TODO) : complete those functions

In [19]:
def predict_pop(last_seen,k):
    return most_popular[:k]

def predict_w2v(last_seen,k):
    return get_similar_ids(w2v,last_seen,k)

#data is list of last_consumed:
def get_predictions(predict_func,data,truth,k=5):
    if k == -1 or k == 0:
        k = num_items
    return [get_relevance_list(predict_func(last_seen,k),will_see) for last_seen,will_see in zip(data,truth)]

**Note**: The `get_predictions(...)` function returns the relevant list associated to predictions

### The following cells should return list of lists

In [20]:
print(get_predictions(predict_pop,last_consumed_item[:5],test_seq[:5],3))
print(get_predictions(predict_w2v,last_consumed_item[:5],test_seq[:5],3))

[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]


expected output: 
```
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]

```

## The return of the MRR and nDCG functions

In [21]:
test_list = [[0,0,1],[0,1,0],[1,0,0],[0,0,0]]

def rr(list_items):
    relevant_indexes = np.asarray(list_items).nonzero()[0]
    
    if len(relevant_indexes) > 0:
        return 1/(relevant_indexes[0]+1) # arrays are indexed from 0
    else:
        return 0

def mrr(list_list_items):
    return np.mean([rr(list_item) for list_item in list_list_items])

mrr(test_list) #0.4583333333333333

# The dcg@k is the sum of the relevance, penalized gradually
def dcg_at_k(r, k):
    """Score is discounted cumulative gain (dcg)
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        
    """
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        
    return 0.

# test values
# r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
# dcg_at_k(r, 1) => 3.0
# dcg_at_k(r, 2) => 4.2618595071429155
r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
print(dcg_at_k(r, 1))
print(dcg_at_k(r, 2))

def mean_dcg(rel_lists,k):
    return np.mean([dcg_at_k(rel_list,k) for rel_list in rel_lists])

# And it's normalized version
def ndcg_at_k(r, k):
    """
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
    """
    dcg_max =  dcg_at_k(sorted(r)[::-1],k) 
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max

# test values
# r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
# ndcg_at_k(r, 1) => 1.0
# ndcg_at_k(r, 4) => 0.794285
    
r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]    
print(ndcg_at_k(r, 4))

def mean_ndcg(rel_lists,k):
    return np.mean([ndcg_at_k(rel_list,k) for rel_list in rel_lists])

3.0
4.2618595071429155
0.7942854176010882


## Let's see how this naïve way of predicting items to show works

In [22]:
pop_preds = get_predictions(predict_pop,last_consumed_item,test_seq,-1)
w2v_preds = get_predictions(predict_w2v,last_consumed_item,test_seq,-1)

print("1/MRR")
print(1/mrr(pop_preds))
print(1/mrr(w2v_preds))
print("")
print("DCG")
print(mean_dcg(pop_preds,5))
print(mean_dcg(w2v_preds,5))
print("")
print("nDCG")
print(mean_ndcg(pop_preds,5))
print(mean_ndcg(w2v_preds,5))

1/MRR
17.872009464417655
11.725264917312641

DCG
0.051054704276776476
0.09450028698336883

nDCG
0.017315723982695274
0.032355614859828866


### (TODO) Can we do better ?

Now, try a different strategy: 


- History should be discarded from prediction
- Instead of basing the prediction on the last seen item, we'll take all the `seen[-n:]` ones (horizon) into account
- To aggregate all items, we'll simply take the min rank to take into account the history offset.
- Equal scores can be handled using the history offset.
Example: 

> Let's say you chose to use the two last seen items `[item 44, Item 398]` to predict the following items

Therefore, using `get_similar_ids` method on both items will yield two lists of **similar** ranked item id's:
 - Similar to item 44: `[item 1, item 33, item 5]`
 - Similar to item 398: `[item 25, item 1, item 5]`
 scores (rank,offset):
 ```
 scores := {item 1: (0,0), item 33: (1,0), item 5: (2,0) , item 25: (0,1)}```
 
 
Then, aggregation by best rank should yield: `[1,25,33,5]`


In [23]:
def predict_max_w2v(seen,k,horizon=2):
    res = [] 
    union = [] 
    for seq in seen[-horizon:] : 
        sim = get_similar_ids(w2v,seq,k) 
        res.append(sim) 
        union += sim 
        union = set(union)
    
    score = dict()
    for item in union:
        
    
    return # a list


w2v_best_preds = get_predictions(predict_max_w2v,train_seq,test_seq,-1)

print(1/mrr(w2v_max_preds))
print(mean_dcg(w2v_max_preds,5))
print(mean_ndcg(w2v_max_preds,5))

IndentationError: expected an indented block (<ipython-input-23-cf26efa212fd>, line 14)

#### => Not really better

## Let's visualize learned embeddings

Just like in the 1st practical, we propose to visualize learnt items embeddings with the [Tensorflow projector](https://projector.tensorflow.org/).

In [ ]:
# This function saves embeddings (a numpy array) and associated labels into tsv files.

def save_embeddings(embs,dict_label,path="saved_word_vectors"):
    """
    embs is Numpy.array(N,size)
    dict_label is {str(word)->int(idx)} or {int(idx)->str(word)}
    """
    def int_first(k,v):
        if type(k) == int:
            return (k,v)
        else:
            return (v,k)

    np.savetxt(f"{path}_vectors.tsv", embs, delimiter="\t")

    #labels 
    if dict_label:
        sorted_labs = np.array([lab for idx,lab in sorted([int_first(k,v) for k,v in dict_label.items()])])
        print(sorted_labs)
        with open(f"{path}_metadata.tsv","w") as metadata_file:
            for x in sorted_labs: #hack for space
                if len(x.strip()) == 0:
                    x = f"space-{len(x)}"
                    
                metadata_file.write(f"{x}\n")

In [ ]:
vec2title = {i:id2title[int(mid)] for i,mid in enumerate(w2v.wv.index2word)}

In [ ]:
save_embeddings(w2v.wv.vectors,vec2title)

## How to:

- Now, [open this link](https://projector.tensorflow.org/), and select "load".
- look for saved_word_vectors_vectors.tsv and saved_word_vectors_metadata.tsv. 

=> These are respectively, the items latent representations and their labels

## Hyperparameters matter when using Word2Vec for Item recommendation:


> Skip-gram with negative sampling, a popular variant of Word2vec originally designed and tuned to create word embeddings for Natural Language Processing, has been used to create item embeddings with successful applications in recommendation. While these fields do not share the same type of data, neither evaluate on the same tasks, recommendation applications tend to use the same already tuned hyperparameters values, even if optimal hyperparameters values are often known to be data and task dependent. We thus investigate the marginal importance of each hyperparameter in a recommendation setting through large hyperparameter grid searches on various datasets. Results reveal that optimizing neglected hyperparameters, namely negative sampling distribution, number of epochs, subsampling parameter and window-size, significantly improves performance on a recommendation task, and can increase it by an order of magnitude. Importantly, we find that optimal hyperparameters configurations for Natural Language Processing tasks and Recommendation tasks are noticeably different. 

[Hyperparameters matter](https://arxiv.org/abs/1804.04212)

#### It turns out that  hyperparameters are really important for this task: especially the sampling parameter.  Try and learn multiple models to see how the ns_exponent parameter modifies the results:


In [ ]:
# the following configuration is the default configuration
w2v = gensim.models.word2vec.Word2Vec(sentences=train_seq_str,
                                size=50, window=3,               ### here we train a cbow model 
                                min_count=0,                      
                                sample=0.001, ns_exponent=-0.4, workers=10,
                                sg=1, hs=0, negative=15,          ### set sg to 1 to train a sg model => Prod2Vec
                                cbow_mean=0,
                                iter=25)



In [ ]:
pop_preds = get_predictions(predict_pop,last_consumed_item,test_seq,-1)
w2v_preds = get_predictions(predict_w2v,last_consumed_item,test_seq,-1)

In [ ]:
print(1/mrr(pop_preds))
print(1/mrr(w2v_preds))

print(mean_dcg(pop_preds,5))
print(mean_dcg(w2v_preds,5))

print(mean_ndcg(pop_preds,5))
print(mean_ndcg(w2v_preds,5))

## Still got time ? Try making a more clever item selection mechanism:

- You could, for example, cluster items in groups (using k-means) and propose the most popular items of the last seen group